# Police nodes


# reddit nodes (NECESSARY, TBD)
## posts
* source: merged_df found below in reddit sections
* node id: post id
* attributes
    * url
    * post title
    * keywords (may be empty)
        * format with ";" delimiter
    * (future ideas: tot comments, upvotes, has media, etc) 

## users (NOT NECESSARY, TBD, NICE TO HAVE)
* source: original cleaned reddit data
* node id: author
* attributes
    * has_posted?
    * has commented?

# nextdoor nodes (NECESSARY, TBD)
* source: nd_keywords_ner.csv
* node_id: post_id
* attributes
    * ShortUrl
    * keywords
    * cannot have post title

# police data nodes (NECESSARY, DONE)
* node_id: incident_id
* attributes
    * priority
    * crime_type (we manually populated)

# Crime corpus nodes (NECESSARY, DONE)
* node: crime type

# Neighborhood corpus nodes (NECESSARY, DONE)
* node: neighborhood location

# Time nodes (DONE)

# Relationships

## Reddit (NOT NECESSARY, nice to have)
* start_id = user id
* end_id = post id
* type: comment, post

## Crime (BELONGS_TO) (NECESSARY)
* start_id = crime post/call (reddit, nextdoor, police data)
* end_id = crime node (crime corpus)
* source type = reddit, nextdoor, police calls (:TYPE)
* time type = time bin
* neighborhood type?

## Crime (HAPPENED_IN) (NECESSARY)
* start_id = crime post/call
* end_id = neighborhood node
* source type = reddit, nextdoor, police calls (:TYPE)

## Crime (HAPPENED_AT) (TBD, nice to have)
* start_id = crime post/call
* end_id = time
* source type = reddit, nextdoor, police calls (:TYPE)

## Other relationships
ethinicity/drugs?/immigration?
General pattern:
start_id: reddit, nextdoor, police
end_id: corpus

In [1]:
# import libraries
from pathlib import Path
import string
import pandas as pd
import sys
sys.path.insert(0, '../data_cleaning_and_integration')

from cleaner_lib import remove_puncuations



In [2]:
# set paths
data_p             = Path("../data")

corpi_p            = data_p / "corpi"
neighborhood_p     = corpi_p / "neighborhood_corpus_binned.csv"
crime_p            = corpi_p / "crime_corpus_binned.csv"

reddit_processed_p = data_p / "processed_reddit_data"
nd_processed_p = data_p / "processed_nextdoor_data"
pd_processed_p = data_p / "processed_pd_data"

# create out path
out_p = data_p / "neo4j_files"
out_p.mkdir(exist_ok=True)

node_p = out_p / "nodes"
node_p.mkdir(exist_ok=True)

relations_p = out_p / "relationships"
relations_p.mkdir(exist_ok=True)


neighborhood_out_p = node_p / "neighborhood_nodes.csv"
crime_out_p = node_p / "crime_nodes.csv"


## Make Corpi Nodes

In [3]:
# read in neighborhood corpus and write to node file
neighborhood_df = pd.read_csv(neighborhood_p)
neighborhood_df.rename(columns = {'neighborhood_set_id':'neighborhood_set_id:ID',
                                 'neighborhood_set':'neighborhood_set:string[]'},inplace=True)
neighborhood_df[":LABEL"] = "neighborhood"
node_neighborhood_df = neighborhood_df[["neighborhood_set_id:ID", "neighborhood_set:string[]", ":LABEL"]].copy()
node_neighborhood_df.drop_duplicates(inplace=True)
node_neighborhood_df.to_csv(neighborhood_out_p, index=False)

In [4]:
# read in neighborhood corpus and write to node file
crime_df = pd.read_csv(crime_p)

crime_df.rename(columns = {'crime_set_id':'crime_set_id:ID',
                                 'crime_set':'crime_set:string[]'},inplace=True)
crime_df[":LABEL"] = "crime"
crime_df = crime_df[["crime_set_id:ID", "crime_set:string[]", ":LABEL"]]
crime_df.drop_duplicates(inplace=True)
crime_df.to_csv(crime_out_p, index=False)

## Reditt Prep

### Reddit: Merge NER and Rake results

In [5]:
ner_p = reddit_processed_p / "cleaned_reddit_ner_12-21_to_1115.csv"
keywords_p = reddit_processed_p / "keyword_extraction.csv"

In [6]:
ner_df = pd.read_csv(ner_p)
print(f"Total observations: {ner_df.shape[0]}")

# drop unamed index
ner_df.drop(columns=['Unnamed: 0'], inplace=True)

ner_df.head()

Total observations: 43421


,subreddit,title,post_id,post_author,post_utc,full_link,post_text,post_text_count,ORG,DATE,EVENT,FAC,GPE,LANGUAGE,LAW,LOC,NORP,PERSON,TIME
0,sandiego,going to visit san diego next week any places...,x4nzh2,Fearmkultra,2022-09-03 06:57:58+00:00,https://www.reddit.com/r/sandiego/comments/x4n...,going to visit san diego next week any places ...,12,['san diego'],['next week'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sandiego,whaley house picture of ghost,x4ntm7,Open_Construction_31,2022-09-03 06:47:09+00:00,https://www.reddit.com/r/sandiego/comments/x4n...,whaley house picture of ghost as a kid i saw t...,199,"['whaley house', 'the whaley house']","['13', '25 yrs ago']",NaN,NaN,['san diegans'],NaN,NaN,NaN,NaN,NaN,"['a minute later', 'late nightearly morning']"
2,sandiego,language exchange,x4n6xv,Poshorock,2022-09-03 06:07:46+00:00,https://www.reddit.com/r/sandiego/comments/x4n...,language exchange is there someone by there wh...,31,NaN,NaN,NaN,NaN,NaN,['english'],NaN,NaN,['spanish'],['san diego'],NaN
3,SanDiegan,chula vista police stopping cars going east on...,x4n5aj,kaptaincorn,2022-09-03 06:04:54+00:00,https://www.reddit.com/r/SanDiegan/comments/x4...,chula vista police stopping cars going east on...,57,NaN,NaN,NaN,NaN,['chula vista'],NaN,NaN,NaN,NaN,NaN,NaN
4,SanDiegan,todd gloria finalizes plan to change park blvd...,x4n2rv,Lemonade_IceCold,2022-09-03 06:00:38+00:00,https://www.reddit.com/r/SanDiegan/comments/x4...,todd gloria finalizes plan to change park blvd...,666,['gtonly'],NaN,NaN,['balboa park'],NaN,NaN,NaN,NaN,['north american'],"['todd gloria', 'kevin']",NaN


In [7]:
ner_df.columns

Index(['subreddit', 'title', 'post_id', 'post_author', 'post_utc', 'full_link',
       'post_text', 'post_text_count', 'ORG', 'DATE', 'EVENT', 'FAC', 'GPE',
       'LANGUAGE', 'LAW', 'LOC', 'NORP', 'PERSON', 'TIME'],
      dtype='object')

In [8]:
keywords_df = pd.read_csv(keywords_p)
print(f"Total observations: {keywords_df.shape[0]}")
keywords_df.drop(columns=['post_text'], inplace=True)
keywords_df.head()

Total observations: 31415


,post_id,keywords
0,x4ntm7,"['suddenly appeared', 'something hard', 'smoke..."
1,x4n6xv,"['language exchange', 'practice spanish', 'pra..."
2,x4n5aj,"['grand ave', 'seen', 'pb', 'holidays', 'end',..."
3,x4n2rv,"['zoo uptown', 'working class', 'traffic elsew..."
4,x4mz7c,"['verbal abuse', 'sell anything', 'extreme win..."


In [9]:
merged_df = ner_df.merge(keywords_df, left_on="post_id", right_on="post_id", how="left")

### Make Reddit nodes based on merged df across ner and keywords

In [10]:
merged_df.head(2)

,subreddit,title,post_id,post_author,post_utc,full_link,post_text,post_text_count,ORG,DATE,EVENT,FAC,GPE,LANGUAGE,LAW,LOC,NORP,PERSON,TIME,keywords
0,sandiego,going to visit san diego next week any places...,x4nzh2,Fearmkultra,2022-09-03 06:57:58+00:00,https://www.reddit.com/r/sandiego/comments/x4n...,going to visit san diego next week any places ...,12,['san diego'],['next week'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sandiego,whaley house picture of ghost,x4ntm7,Open_Construction_31,2022-09-03 06:47:09+00:00,https://www.reddit.com/r/sandiego/comments/x4n...,whaley house picture of ghost as a kid i saw t...,199,"['whaley house', 'the whaley house']","['13', '25 yrs ago']",NaN,NaN,['san diegans'],NaN,NaN,NaN,NaN,NaN,"['a minute later', 'late nightearly morning']","['suddenly appeared', 'something hard', 'smoke..."


### Reddit Nodes

In [11]:
reddit_post_node_file = node_p / 'reddit_post_nodes.csv'

merged_df['post_id:ID'] = merged_df['post_id']
reddit_post_nodes_df = merged_df[['post_id:ID','full_link','title','keywords']].copy()

reddit_post_nodes_df.fillna("", inplace=True)
reddit_post_nodes_df.keywords = reddit_post_nodes_df.keywords.str.replace('[', '')
reddit_post_nodes_df.keywords = reddit_post_nodes_df.keywords.str.replace(']', '')
reddit_post_nodes_df.keywords = reddit_post_nodes_df.keywords.apply(lambda x: ';'.join(x.split(",")))
reddit_post_nodes_df[':LABEL'] = 'POST'

#Removing duplicate post ids.
# Around 352 rows had duplicate post_id
reddit_post_nodes_df = reddit_post_nodes_df[~reddit_post_nodes_df['post_id:ID'].duplicated()]
print("shape of reddit_post_nodes_df", reddit_post_nodes_df.shape)
display(reddit_post_nodes_df)

reddit_post_nodes_df.to_csv(reddit_post_node_file,index=False)

shape of reddit_post_nodes_df (43273, 5)


/var/folders/jg/d8_zwblj5ql38q843xmy7h9r0000gp/T/ipykernel_77152/502371629.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  reddit_post_nodes_df.keywords = reddit_post_nodes_df.keywords.str.replace('[', '')
/var/folders/jg/d8_zwblj5ql38q843xmy7h9r0000gp/T/ipykernel_77152/502371629.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  reddit_post_nodes_df.keywords = reddit_post_nodes_df.keywords.str.replace(']', '')


,post_id:ID,full_link,title,keywords,:LABEL
0,x4nzh2,https://www.reddit.com/r/sandiego/comments/x4n...,going to visit san diego next week any places...,,POST
1,x4ntm7,https://www.reddit.com/r/sandiego/comments/x4n...,whaley house picture of ghost,'suddenly appeared'; 'something hard'; 'smoke ...,POST
2,x4n6xv,https://www.reddit.com/r/sandiego/comments/x4n...,language exchange,'language exchange'; 'practice spanish'; 'prac...,POST
3,x4n5aj,https://www.reddit.com/r/SanDiegan/comments/x4...,chula vista police stopping cars going east on...,'grand ave'; 'seen'; 'pb'; 'holidays'; 'end'; ...,POST
4,x4n2rv,https://www.reddit.com/r/SanDiegan/comments/x4...,todd gloria finalizes plan to change park blvd...,'zoo uptown'; 'working class'; 'traffic elsewh...,POST
...,...,...,...,...,...
43620,scdqum,https://www.reddit.com/r/UCSD/comments/scdqum/...,la jolla donor makes 50m research t that could...,'wasnt aware'; 'san diego'; 'never wondered'; ...,POST
43621,sca7fv,https://www.reddit.com/r/UCSD/comments/sca7fv/...,new covid variant detected in at least 40 diff...,'sigma variant'; 'new shot'; 'like omicron'; '...,POST
43622,sc9b5t,https://www.reddit.com/r/sandiego/comments/sc9...,tmz baltimore maggots leaked video twitter sca...,,POST
43623,sc90i4,https://www.reddit.com/r/UCSD/comments/sc90i4/...,mailing services while school’s online,'thing thankfully'; 'theyre forwarding'; 'pret...,POST


Reddit happened_in relationship
* crime related post happened_in neighborhood

### Nextdoor Nodes

In [12]:
# source: ../data/processed_nextdoor_data/nd_keywords_ner.csv
nd_merged_df = pd.read_csv(nd_processed_p / "nd_keywords_ner.csv")
nd_merged_df.head()

,post_id,ShortLink,Author,post_text,post_text_count,Neighborhood,PERSON,TIME,DATE,ORG,...,GPE,FAC,LOC,LAW,LANGUAGE,EVENT,keywords,crime_score,ethnicity_score,neighborhood_score
0,nd1,https://nextdoor.com/p/--3jc5nsXN58?view=detail,Hannah Lopez,how late can people be working on construction...,131,Corridor,tapebill,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"['willful violation', 'news trying', 'means ca...",0.005391,0.0,0.000000
1,nd2,https://nextdoor.com/p/--mjpdwdS3yx?view=detail,Tim Welch,rain has finally arrived in north park but las...,280,Montclair,"['chad jeremy 1964yeah', 'nicolas cage']",only 3 minutes,"['tomorrow', 'yesterday', 'about two months la...",like.humidity,...,"['china', 'san miguel de allende']",NaN,NaN,NaN,NaN,NaN,"['“ yeah', 'vehicles chance', 'shall rebuild',...",0.000000,0.0,0.002079
2,nd3,https://nextdoor.com/p/-3GwdKj4_sMm?view=detail,News,dont we have a water shortage... jennifer that...,1250,NaN,"['jennifer', 'zanyface', 'agendawalter', 'wate...",NaN,"['a day', '2 years ago', '5000 a month', '13',...","['sandags series', 'angelescarol dellangela']",...,"['san francisco', 'san diego', 'differently.go...",NaN,NaN,NaN,NaN,NaN,"['… enough', 'water usage', 'water situation',...",0.008448,0.0,0.000000
3,nd4,https://nextdoor.com/p/-4qn3_2yNk_Y?view=detail,Frank Negrete,guess nd didnt like my question about drinking...,82,Hillcrest Northeast,"['ndi’d', 'moderatorselectra hendrickson']",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"['public facewithtearsofjoy', 'faces bios', 'd...",0.000000,0.0,0.000000
4,nd5,https://nextdoor.com/p/-5-J-BXgJ84y?view=detail,Dawn Burton,day time robbery marston hillsupdate. update u...,1853,Hillcrest Southeast,"['max', 'insanitylaurie hewitt', 'pam lauri', ...","['530 pm', 'morning', 'night', 'around midnigh...","['a month ago', 'age 2030', 'feb 26', 'about t...","['marston', 'nextdoor wvideo', 'dogood', 'your...",...,"['california', 'california', 'essex st', 'verm...",NaN,NaN,NaN,NaN,NaN,"['yet nothing', 'violent felonies', 'unlawful ...",0.042534,0.0,0.000000


In [14]:
nd_post_node_file = node_p / 'nd_post_nodes.csv'

nd_merged_df['post_id:ID'] = nd_merged_df['post_id']
nd_post_nodes_df = nd_merged_df[['post_id:ID','ShortLink','keywords']].copy()

nd_post_nodes_df.fillna("", inplace=True)
nd_post_nodes_df.keywords = nd_post_nodes_df.keywords.str.replace('[', '')
nd_post_nodes_df.keywords = nd_post_nodes_df.keywords.str.replace(']', '')
nd_post_nodes_df.keywords = nd_post_nodes_df.keywords.apply(lambda x: ';'.join(x.split(",")))
nd_post_nodes_df[':LABEL'] = 'POST'

print("shape of nd_post_nodes_df", nd_post_nodes_df.shape)
display(nd_post_nodes_df)

nd_post_nodes_df.to_csv(nd_post_node_file,index=False)

shape of nd_post_nodes_df (2808, 4)


/var/folders/jg/d8_zwblj5ql38q843xmy7h9r0000gp/T/ipykernel_77152/2006162221.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nd_post_nodes_df.keywords = nd_post_nodes_df.keywords.str.replace('[', '')
/var/folders/jg/d8_zwblj5ql38q843xmy7h9r0000gp/T/ipykernel_77152/2006162221.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nd_post_nodes_df.keywords = nd_post_nodes_df.keywords.str.replace(']', '')


,post_id:ID,ShortLink,keywords,:LABEL
0,nd1,https://nextdoor.com/p/--3jc5nsXN58?view=detail,'willful violation'; 'news trying'; 'means cap...,POST
1,nd2,https://nextdoor.com/p/--mjpdwdS3yx?view=detail,'“ yeah'; 'vehicles chance'; 'shall rebuild'; ...,POST
2,nd3,https://nextdoor.com/p/-3GwdKj4_sMm?view=detail,'… enough'; 'water usage'; 'water situation'; ...,POST
3,nd4,https://nextdoor.com/p/-4qn3_2yNk_Y?view=detail,'public facewithtearsofjoy'; 'faces bios'; 'de...,POST
4,nd5,https://nextdoor.com/p/-5-J-BXgJ84y?view=detail,'yet nothing'; 'violent felonies'; 'unlawful b...,POST
...,...,...,...,...
2803,nd2817,https://nextdoor.com/p/zyBKcPsfG8p4?view=detail,'xxx amount'; 'vacation home'; 'uspspaula abso...,POST
2804,nd2818,https://nextdoor.com/p/zzWdg8FDxMw4?view=detail,'sketchy scammy'; 'senders email'; 'scammichae...,POST
2805,nd2819,https://nextdoor.com/p/zzYsgLb5T2sb?view=detail,'‘ charlie'; 'yrs old'; 'outcarol thank'; 'hi ...,POST
2806,nd2820,https://nextdoor.com/p/zzgTmx49yTM4?view=detail,'tongueincheekdarn fireworks'; 'seconds apart'...,POST


## Extracting Ethnicity for creating nodes

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
ethnicity_corpus = pd.read_csv(corpi_p / 'ethnicity_corpus.csv')
ethnicity_corpus.head()
ethnicity_vectorizer = CountVectorizer(vocabulary = ethnicity_corpus['ethnicity'])
ethnicity_vectorizer_matches = ethnicity_vectorizer.transform(merged_df['post_text'])
ethnicity_exact_matches_df = pd.DataFrame({'dfindex': ethnicity_vectorizer_matches.nonzero()[0], 
                                      'ethnicityindex': ethnicity_vectorizer_matches.nonzero()[1]})
ethnicity_exact_matches_df = ethnicity_exact_matches_df.merge(ethnicity_corpus, how = 'inner', 
                                                                    left_on = 'ethnicityindex', 
                                                                    right_index = True)
ethnicity_exact_matches_df = ethnicity_exact_matches_df.groupby(by = 'dfindex', 
                                                                      as_index = False).agg({'ethnicity': lambda x: x.tolist()})
ethnicity_exact_matches_df

,dfindex,ethnicity
0,2,"[english, spanish]"
1,4,[american]
2,47,[black]
3,60,[white]
4,88,[black]
...,...,...
2444,43520,[white]
2445,43561,[italy]
2446,43581,[white]
2447,43619,[german]


### Police Calls data nodes

Police Crime nodes

In [17]:
# Police Crime nodes

pd_df = pd.read_csv(pd_processed_p / 'merged_pd_data.csv')

##Change priority to int
pd_df['priority'] = pd_df['priority'].astype(int)
#droping duplicates
pd_df.drop_duplicates(inplace=True)

pd_df[":LABEL"] = "REPORTED_CRIME"

pd_df['incident_num:ID'] = pd_df['incident_num']

#re-arranging columns
pd_crime_node_df = pd_df[['incident_num:ID','priority','crime_type',':LABEL']]

pd_crime_node_df.to_csv(node_p / 'pd_crime_nodes.csv',index=False)


### (Police incident) -happened_in-> (neighborhood)

In [18]:
neh_id_dict = dict(zip(neighborhood_df.neighborhood, neighborhood_df['neighborhood_set_id:ID']))


In [19]:
def get_neh_id(neh):
    '''This returns the neighborhood id from neighborhood'''
    try:
        return neh_id_dict[neh.lower()]
    except:
        return 'unknown'

In [20]:
pd_happened_in_rel_df = pd_df.copy()

# Looks like the initial corpus was create by replacing chars with space
pd_happened_in_rel_df["neighborhood"] = pd_happened_in_rel_df["neighborhood"].replace('\'', '', regex=True)
pd_happened_in_rel_df["neighborhood"] = pd_happened_in_rel_df["neighborhood"].replace('/', ' ', regex=True)
pd_happened_in_rel_df["neighborhood"] = pd_happened_in_rel_df["neighborhood"].replace('-', ' ', regex=True)

pd_happened_in_rel_df = pd_happened_in_rel_df.rename(columns={'incident_num':'incident_num:START_ID'})
non_na_neh = pd.notna(pd_happened_in_rel_df['neighborhood'])
pd_happened_in_rel_df = pd_happened_in_rel_df[non_na_neh]
pd_happened_in_rel_df = remove_puncuations(pd_happened_in_rel_df,'neighborhood')
pd_happened_in_rel_df[':END_ID'] = pd_happened_in_rel_df['neighborhood'].apply(lambda x: get_neh_id(x))
pd_happened_in_rel_df[':TYPE'] = 'HAPPENED_IN'
pd_happened_in_rel_df = pd_happened_in_rel_df[pd_happened_in_rel_df[':END_ID']!='unknown']
pd_happened_in_rel_df = pd_happened_in_rel_df[['incident_num:START_ID',':END_ID',':TYPE']]
pd_happened_in_rel_df.to_csv(relations_p / 'police_HI_rels.csv',index=False)


In [21]:
pd_df[pd_df['incident_num:ID']=='E22010000006']

,incident_num,date_time,day_of_week,address_number_primary,address_dir_primary,address_road_primary,address_sfx_primary,address_dir_intersecting,address_road_intersecting,address_sfx_intersecting,...,disposition,beat,priority,neighborhood,call_type_desc,crime_type,crime_focus,date_time_bin,:LABEL,incident_num:ID
0,E22010000006,2022-01-01 00:01:14,7,4600,NaN,ZION,AVE,NaN,NaN,NaN,...,K,321,2,Grantville,DISTURBING PEACE,minor,person,Late Night,REPORTED_CRIME,E22010000006


### REDDIT POST -HAPPENED_IN-> NEIGHBORHOOD

In [23]:
import re
def remove_serialized_list_chars(x):
    return re.sub("\[|\]|'", "", x).split(",")
    

In [24]:
#reddit neighborhood
reddit_crime_neighborhood_df = pd.read_csv(reddit_processed_p / 'reddit_crime_neighborhood.csv')

#ignoring all the recs having empty list
reddit_crime_neighborhood_df = reddit_crime_neighborhood_df[reddit_crime_neighborhood_df['neighborhood']!='[]']

#Since on disk the lists were saved in string representation, need to de-serialize it again
reddit_crime_neighborhood_df['neighborhood'] = reddit_crime_neighborhood_df['neighborhood'].apply(lambda x: remove_serialized_list_chars(x))
reddit_crime_neighborhood_df = reddit_crime_neighborhood_df.explode(column='neighborhood')

#rename column
reddit_crime_neighborhood_df['post_id:START_ID'] = reddit_crime_neighborhood_df['post_id']

#looking up the corpus file to get the correct END ID
reddit_crime_neighborhood_df[':END_ID'] = reddit_crime_neighborhood_df['neighborhood'].apply(lambda x: get_neh_id(x.strip()))
reddit_crime_neighborhood_df[':TYPE'] = 'HAPPENED_IN'

#Join to make sure I pick only pos _id from the nodes
reddit_crime_neighborhood_df = pd.merge(reddit_post_nodes_df, reddit_crime_neighborhood_df, left_on='post_id:ID', right_on='post_id:START_ID')


reddit_crime_neighborhood_df = reddit_crime_neighborhood_df[['post_id:START_ID',':END_ID',':TYPE']]
reddit_crime_neighborhood_df.to_csv(relations_p / 'reddit_HI_rels.csv',index=False)

### NEXTDOOR POST -HAPPENED_IN-> NEIGHBORHOOD

In [25]:
#nextdoor neighborhood
nd_crime_neighborhood_df = pd.read_csv( nd_processed_p / 'transitionary_files/nd_neighborhoods_matches.csv')
nd_crime_neighborhood_df = nd_crime_neighborhood_df[nd_crime_neighborhood_df['neighborhood']!='[]']
nd_crime_neighborhood_df['neighborhood'] = nd_crime_neighborhood_df['neighborhood'].apply(lambda x: remove_serialized_list_chars(x))
nd_crime_neighborhood_df = nd_crime_neighborhood_df.explode(column='neighborhood')
#rename column
nd_crime_neighborhood_df['post_id:START_ID'] = nd_crime_neighborhood_df['post_id']
nd_crime_neighborhood_df[':END_ID'] = nd_crime_neighborhood_df['neighborhood'].apply(lambda x: get_neh_id(x.strip()))
nd_crime_neighborhood_df[':TYPE'] = 'HAPPENED_IN'

#Join to make sure I pick only pos _id from the nodes
nd_crime_neighborhood_df = pd.merge(nd_post_nodes_df, nd_crime_neighborhood_df, left_on='post_id:ID', right_on='post_id:START_ID')

nd_crime_neighborhood_df = nd_crime_neighborhood_df[['post_id:START_ID',':END_ID',':TYPE']]
nd_crime_neighborhood_df.to_csv(relations_p / 'nextdoor_HI_rels.csv',index=False)
